In [1]:
!pip install torch==1.12.1 torchvision==0.13.1 -f https://download.pytorch.org/whl/cpu/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires torch>=1.13.0, but you have torch 1.12.1+cpu which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 1.12.1+cpu which is incompatible.


In [2]:
# import packages
import time
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.optim as optim

from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

from google.colab import files
import io

warnings.filterwarnings(action='ignore')

In [3]:
uploaded = files.upload()

Saving wdata.csv to wdata.csv


In [4]:
train = pd.read_csv(io.BytesIO(uploaded['wdata.csv']))
#train, valid = train_test_split(train, test_size = 0.3, random_state = 55)

In [5]:
print(train.head(5))
train.info()
train.isnull().sum()

#Cloud Cover, Weather Type String -> Int Mapping
mapping = {'clear': 1, 'partly cloudy': 2, 'cloudy': 3, 'overcast': 4}
train['Cloud Cover'] = train['Cloud Cover'].map(mapping)

mapping = {'Sunny': 1, 'Cloudy': 2, 'Rainy': 3, 'Snowy': 4}
train['Weather Type'] = train['Weather Type'].map(mapping)
print(train.head(5))

   Temperature  Humidity  Wind Speed  Precipitation (%)    Cloud Cover  \
0         14.0        73         9.5               82.0  partly cloudy   
1         39.0        96         8.5               71.0  partly cloudy   
2         30.0        64         7.0               16.0          clear   
3         38.0        83         1.5               82.0          clear   
4         27.0        74        17.0               66.0       overcast   

   Atmospheric Pressure  UV Index  Season  Visibility (km)  Location  \
0               1010.82         2  Winter              3.5    inland   
1               1011.43         7  Spring             10.0    inland   
2               1018.72         5  Spring              5.5  mountain   
3               1026.25         7  Spring              1.0   coastal   
4                990.67         1  Winter              2.5  mountain   

  Weather Type  
0        Rainy  
1       Cloudy  
2        Sunny  
3        Sunny  
4        Rainy  
<class 'pandas.core.

In [6]:
# Location OneHotEncoder 사용
enc = OneHotEncoder(sparse_output=False, dtype=int)
enc.fit(train[['Location']])

tmp = pd.DataFrame(enc.transform(train[['Location']]), columns=enc.get_feature_names_out(['Location']))
train = pd.concat([train, tmp], axis=1)
train = train.drop(columns='Location')

# 결과 확인
print(train.head(3))

   Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
0         14.0        73         9.5               82.0            2   
1         39.0        96         8.5               71.0            2   
2         30.0        64         7.0               16.0            1   

   Atmospheric Pressure  UV Index  Season  Visibility (km)  Weather Type  \
0               1010.82         2  Winter              3.5             3   
1               1011.43         7  Spring             10.0             2   
2               1018.72         5  Spring              5.5             1   

   Location_coastal  Location_inland  Location_mountain  
0                 0                1                  0  
1                 0                1                  0  
2                 0                0                  1  


In [7]:
# 기압 Column 정규화
feature_columns = ['Atmospheric Pressure']

train[feature_columns] = (train[feature_columns] - train[feature_columns].mean()) / train[feature_columns].std()
train[feature_columns] = train[feature_columns] + 1

# 정규화된 데이터 확인
print(train.head())

   Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
0         14.0        73         9.5               82.0            2   
1         39.0        96         8.5               71.0            2   
2         30.0        64         7.0               16.0            1   
3         38.0        83         1.5               82.0            1   
4         27.0        74        17.0               66.0            4   

   Atmospheric Pressure  UV Index  Season  Visibility (km)  Weather Type  \
0              1.134198         2  Winter              3.5             3   
1              1.150596         7  Spring             10.0             2   
2              1.346566         5  Spring              5.5             1   
3              1.548987         7  Spring              1.0             1   
4              0.592525         1  Winter              2.5             3   

   Location_coastal  Location_inland  Location_mountain  
0                 0                1                

In [ ]:
#모든 Columns 사용해서 기온 예측을 위한 전처리 및 모델 정의


data_copy = train.copy()

# Step 2: Season Column 원핫 인코딩
encoder = OneHotEncoder(sparse_output=False)
season_encoded = encoder.fit_transform(data_copy[['Season']])
season_encoded_df = pd.DataFrame(season_encoded, columns=encoder.get_feature_names_out(['Season']))

data_processed = pd.concat([data_copy.drop(columns=['Season']), season_encoded_df], axis=1)
train_data, val_data = train_test_split(data_processed, test_size=0.2, random_state=42)

class TemperatureDataset(Dataset):
    def __init__(self, data):
        self.X = torch.tensor(data.drop(columns=['Temperature']).values, dtype=torch.float32)
        self.y = torch.tensor(data['Temperature'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TemperatureDataset(train_data)
val_dataset = TemperatureDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

class TemperatureNN(nn.Module):
    def __init__(self):
        super(TemperatureNN, self).__init__()
        self.fc1 = nn.Linear(15, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [ ]:
#모든 Columns 사용해서 기온 예측 살행

model = TemperatureNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)  # MSE 계산
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)  # 평균 계산

    # 검증 루프
    model.eval()
    val_loss = 0
    num_batches = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)  # MSE 계산
            val_loss += loss.item()  # 배치 손실 누적
    val_loss /= len(val_loader)  # 검증 손실 평균 계산

    # 출력
    if (epoch + 1) % 20 == 0:
      print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

print(f"Total Training Time: {time.time() - start_time:.2f} seconds")

Epoch 20/200, Train Loss: 120.3877, Validation Loss: 110.3380
Epoch 40/200, Train Loss: 112.1971, Validation Loss: 114.0968
Epoch 60/200, Train Loss: 98.6839, Validation Loss: 121.6458
Epoch 80/200, Train Loss: 88.3935, Validation Loss: 130.4535
Epoch 100/200, Train Loss: 81.4942, Validation Loss: 134.0410
Epoch 120/200, Train Loss: 76.0607, Validation Loss: 136.9552
Epoch 140/200, Train Loss: 70.6241, Validation Loss: 132.7450
Epoch 160/200, Train Loss: 67.3696, Validation Loss: 137.8783
Epoch 180/200, Train Loss: 61.0310, Validation Loss: 140.4060
Epoch 200/200, Train Loss: 59.9666, Validation Loss: 147.7364
Total Training Time: 300.48 seconds


In [ ]:
#모든 Columns 사용해서 계절 예측을 위한 전처리 및 모델 정의


data_copy = train.copy()

# Season 문자열을 정수로 인코딩
label_encoder = LabelEncoder()
data_copy['Season'] = label_encoder.fit_transform(data_copy['Season'])  # Spring=0, Summer=1, Autumn=2, Winter=3

# 데이터셋 분리
season_train_data, season_val_data = train_test_split(data_copy, test_size=0.2, random_state=42)

class SeasonDataset(Dataset):
    def __init__(self, data):
        self.X = torch.tensor(data.drop(columns=['Season']).values, dtype=torch.float32)
        self.y = torch.tensor(data['Season'].values, dtype=torch.long)  # 정수형 라벨

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

season_train_dataset = SeasonDataset(season_train_data)
season_val_dataset = SeasonDataset(season_val_data)

season_train_loader = DataLoader(season_train_dataset, batch_size=16, shuffle=True)
season_val_loader = DataLoader(season_val_dataset, batch_size=16, shuffle=False)

class SeasonClassifier(nn.Module):
    def __init__(self):
        super(SeasonClassifier, self).__init__()
        self.fc1 = nn.Linear(12, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [ ]:
#모든 Columns 사용해서 계절 예측 실행

model = SeasonClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 훈련 및 검증 루프
start_time = time.time()
for epoch in range(200):
    # 훈련 루프
    model.train()
    total_train_loss = 0
    total_train_correct = 0
    total_train_samples = 0

    for inputs, targets in season_train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)  # 배치 손실 계산
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 누적
        total_train_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        total_train_correct += (preds == targets).sum().item()
        total_train_samples += targets.size(0)

    avg_train_loss = total_train_loss / len(train_loader)
    train_accuracy = total_train_correct / total_train_samples

    # 검증 루프
    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    total_val_samples = 0

    with torch.no_grad():
        for inputs, targets in season_val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)  # 배치 손실 계산

            # 손실 및 정확도 누적
            total_val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            total_val_correct += (preds == targets).sum().item()
            total_val_samples += targets.size(0)

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = total_val_correct / total_val_samples

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

print(f"Total Training Time: {time.time() - start_time:.2f} seconds")

Epoch 20, Train Loss: 1.0807, Train Accuracy: 0.4253, Validation Loss: 1.0535, Validation Accuracy: 0.4303
Epoch 40, Train Loss: 1.0685, Train Accuracy: 0.4325, Validation Loss: 1.0790, Validation Accuracy: 0.4318
Epoch 60, Train Loss: 1.0500, Train Accuracy: 0.4430, Validation Loss: 1.1405, Validation Accuracy: 0.4277
Epoch 80, Train Loss: 1.0413, Train Accuracy: 0.4432, Validation Loss: 1.1771, Validation Accuracy: 0.4265
Epoch 100, Train Loss: 1.0332, Train Accuracy: 0.4515, Validation Loss: 1.1943, Validation Accuracy: 0.4246
Epoch 120, Train Loss: 1.0269, Train Accuracy: 0.4545, Validation Loss: 1.2893, Validation Accuracy: 0.4265
Epoch 140, Train Loss: 1.0211, Train Accuracy: 0.4575, Validation Loss: 1.3109, Validation Accuracy: 0.4288
Epoch 160, Train Loss: 1.0252, Train Accuracy: 0.4566, Validation Loss: 1.3395, Validation Accuracy: 0.4261
Epoch 180, Train Loss: 1.0212, Train Accuracy: 0.4589, Validation Loss: 1.4180, Validation Accuracy: 0.4265
Epoch 200, Train Loss: 1.0289, T

In [ ]:
#각 계절 별 기온 예측 모델 생성

class SeasonalTemperatureNN(nn.Module):
    def __init__(self):
        super(SeasonalTemperatureNN, self).__init__()
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)  # 출력 크기 1 (Temperature)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

# 계절 리스트
seasons = ["Spring", "Summer", "Autumn", "Winter"]

# 데이터 분리 및 로더 생성
season_data = {}  # 데이터 저장 딕셔너리
train_loaders = {}  # 학습용 데이터 로더
val_loaders = {}  # 검증용 데이터 로더
models = {}  # 계절별 모델 저장
optimizers = {}  # 계절별 옵티마이저 저장

for season in seasons:
    # 데이터 분리
    season_subset = train[train['Season'] == season].drop(columns=['Season'])
    train_data, val_data = train_test_split(season_subset, test_size=0.2, random_state=42)

    # 데이터 저장
    season_data[season] = {"train": train_data, "val": val_data}

    # 데이터 로더 생성
    train_loaders[season] = DataLoader(TemperatureDataset(train_data), batch_size=16, shuffle=True)
    val_loaders[season] = DataLoader(TemperatureDataset(val_data), batch_size=16, shuffle=False)

    # 계절별 모델 및 옵티마이저 초기화
    models[season] = SeasonalTemperatureNN()
    optimizers[season] = optim.Adam(models[season].parameters(), lr=0.001)

In [ ]:
print(season_data["Winter"]["train"].head(5))

       Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
1606          58.0        76        26.5               97.0            2   
6961          -4.0        93         7.0               53.0            4   
8193          43.0        67         4.0                0.0            2   
6641          55.0        79         8.0               92.0            2   
11352         -8.0        66        17.5               91.0            4   

       Atmospheric Pressure  UV Index  Visibility (km)  Weather Type  \
1606               1.017530         7              7.0             2   
6961               0.799248         0              5.0             4   
8193               1.403287         9              6.0             1   
6641               1.152746        10              1.0             2   
11352              0.475320         0              1.0             4   

       Location_coastal  Location_inland  Location_mountain  
1606                  1                0        

In [ ]:
#모든 Columns 사용해서 계절 예측 실행

num_epochs = 200  # 학습 에포크 수
losses = {}  # 최종 손실 저장 딕셔너리

# 각 계절별 학습 및 검증 루프
start_time = time.time()
for season in seasons:
    print(f"Training {season} Model...")

    # 계절별 모델, 옵티마이저, 데이터 로더
    model = models[season]
    optimizer = optimizers[season]
    train_loader = train_loaders[season]
    val_loader = val_loaders[season]
    criterion = nn.MSELoss()  # 손실 함수

    for epoch in range(num_epochs):
        #학습 단계
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)  # 손실 계산
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        avg_train_loss = total_train_loss / len(train_loader)

        #검증 단계
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs.squeeze(), targets)  # 검증 손실 계산
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_loader)

        # 에포크별 출력
        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

    # 최종 손실 저장
    losses[season] = {"train_loss": avg_train_loss, "val_loss": avg_val_loss}

print(f"Total Training Time: {time.time() - start_time:.2f} seconds")


Training Spring Model...
Epoch 20/200 - Train Loss: 170.1662, Validation Loss: 147.3767
Epoch 40/200 - Train Loss: 151.3903, Validation Loss: 140.1207
Epoch 60/200 - Train Loss: 135.5075, Validation Loss: 151.7140
Epoch 80/200 - Train Loss: 121.3672, Validation Loss: 155.7047
Epoch 100/200 - Train Loss: 106.6319, Validation Loss: 179.1522
Epoch 120/200 - Train Loss: 101.5646, Validation Loss: 175.5712
Epoch 140/200 - Train Loss: 87.4820, Validation Loss: 169.6328
Epoch 160/200 - Train Loss: 83.7999, Validation Loss: 188.2219
Epoch 180/200 - Train Loss: 71.1662, Validation Loss: 203.8723
Epoch 200/200 - Train Loss: 68.8447, Validation Loss: 173.5537
Training Summer Model...
Epoch 20/200 - Train Loss: 168.2167, Validation Loss: 174.3179
Epoch 40/200 - Train Loss: 150.9192, Validation Loss: 166.6802
Epoch 60/200 - Train Loss: 137.7925, Validation Loss: 168.8644
Epoch 80/200 - Train Loss: 134.5208, Validation Loss: 161.8928
Epoch 100/200 - Train Loss: 119.7203, Validation Loss: 161.6264
Ep

In [ ]:
#기온, 습도, 풍속 만을 사용해서 계절 예측을 위한 전처리 및 단순한 모델 정의

data_copy = train.copy()

# Season 문자열을 정수로 인코딩
label_encoder = LabelEncoder()
data_copy['Season'] = label_encoder.fit_transform(data_copy['Season'])  # Spring=0, Summer=1, Autumn=2, Winter=3

print(data_copy.head(5))

# 데이터셋 분리
season_train_data, season_val_data = train_test_split(data_copy, test_size=0.2, random_state=42)

class SeasonDataset(Dataset):
    def __init__(self, data):
        self.X = torch.tensor(data[['Temperature', 'Humidity', 'Wind Speed']].values, dtype=torch.float32)
        self.y = torch.tensor(data['Season'].values, dtype=torch.long)  # 정수형 라벨

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 데이터셋 생성
season_train_dataset = SeasonDataset(season_train_data)
season_val_dataset = SeasonDataset(season_val_data)

# 데이터 로더 생성
season_train_loader = DataLoader(season_train_dataset, batch_size=16, shuffle=True)
season_val_loader = DataLoader(season_val_dataset, batch_size=16, shuffle=False)

class SeasonClassifier(nn.Module):
    def __init__(self):
        super(SeasonClassifier, self).__init__()
        self.fc1 = nn.Linear(3, 6)
        self.fc2 = nn.Linear(6, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

   Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
0         14.0        73         9.5               82.0            2   
1         39.0        96         8.5               71.0            2   
2         30.0        64         7.0               16.0            1   
3         38.0        83         1.5               82.0            1   
4         27.0        74        17.0               66.0            4   

   Atmospheric Pressure  UV Index  Season  Visibility (km)  Weather Type  \
0              1.134198         2       3              3.5             3   
1              1.150596         7       1             10.0             2   
2              1.346566         5       1              5.5             1   
3              1.548987         7       1              1.0             1   
4              0.592525         1       3              2.5             3   

   Location_coastal  Location_inland  Location_mountain  
0                 0                1                

In [ ]:
#기온, 습도, 풍속 만을 사용해서 계절 예측 실행

model = SeasonClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# 훈련 및 검증 루프
start_time = time.time()
for epoch in range(200):
    # 훈련 루프
    model.train()
    total_train_loss = 0
    total_train_correct = 0
    total_train_samples = 0

    for inputs, targets in season_train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)  # 배치 손실 계산
        loss.backward()
        optimizer.step()

        # 손실 및 정확도 누적
        total_train_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        total_train_correct += (preds == targets).sum().item()
        total_train_samples += targets.size(0)

    avg_train_loss = total_train_loss / len(season_train_loader)
    train_accuracy = total_train_correct / total_train_samples

    # 검증 루프
    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    total_val_samples = 0

    with torch.no_grad():
        for inputs, targets in season_val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)  # 배치 손실 계산

            # 손실 및 정확도 누적
            total_val_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            total_val_correct += (preds == targets).sum().item()
            total_val_samples += targets.size(0)

    avg_val_loss = total_val_loss / len(season_val_loader)
    val_accuracy = total_val_correct / total_val_samples

    # 에포크별 출력
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

print(f"Total Training Time: {time.time() - start_time:.2f} seconds")

Epoch 20, Train Loss: 1.1585, Train Accuracy: 0.4211, Validation Loss: 1.1308, Validation Accuracy: 0.4292
Epoch 40, Train Loss: 1.1525, Train Accuracy: 0.4212, Validation Loss: 1.1249, Validation Accuracy: 0.4277
Epoch 60, Train Loss: 1.1504, Train Accuracy: 0.4218, Validation Loss: 1.1226, Validation Accuracy: 0.4326
Epoch 80, Train Loss: 1.1490, Train Accuracy: 0.4224, Validation Loss: 1.1231, Validation Accuracy: 0.4307
Epoch 100, Train Loss: 1.1482, Train Accuracy: 0.4273, Validation Loss: 1.1238, Validation Accuracy: 0.4314
Epoch 120, Train Loss: 1.1475, Train Accuracy: 0.4254, Validation Loss: 1.1363, Validation Accuracy: 0.4284
Epoch 140, Train Loss: 1.1453, Train Accuracy: 0.4303, Validation Loss: 1.1251, Validation Accuracy: 0.4254
Epoch 160, Train Loss: 1.1444, Train Accuracy: 0.4230, Validation Loss: 1.1157, Validation Accuracy: 0.4375
Epoch 180, Train Loss: 1.1421, Train Accuracy: 0.4202, Validation Loss: 1.1102, Validation Accuracy: 0.4345
Epoch 200, Train Loss: 1.1369, T

In [ ]:
#기온, 습도, 풍속 만을 사용해서 계절 예측을 위한 전처리 및 Decision Tree 실행


# 입력 데이터와 타겟 분리
X_train = season_train_data[['Temperature', 'Humidity', 'Wind Speed']]
y_train = season_train_data['Season']

X_val = season_val_data[['Temperature', 'Humidity', 'Wind Speed']]
y_val = season_val_data['Season']

# 모델 초기화
decision_tree = DecisionTreeClassifier(criterion='gini', max_depth=2, random_state=42)

# 모델 학습
decision_tree.fit(X_train, y_train)

# 검증 데이터 예측
y_pred = decision_tree.predict(X_val)

# 정확도 계산
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")

# 평가 출력
print(classification_report(y_val, y_pred))

Validation Accuracy: 0.4462
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       475
           1       0.26      0.80      0.40       521
           2       0.00      0.00      0.00       507
           3       0.72      0.67      0.69      1137

    accuracy                           0.45      2640
   macro avg       0.25      0.37      0.27      2640
weighted avg       0.36      0.45      0.38      2640



In [11]:
#season 제외한 Columns 사용해서 기온 예측을 위한 전처리 및 모델 정의

data_copy = train.copy()


data_processed = data_copy
train_data, val_data = train_test_split(data_processed, test_size=0.2, random_state=42)

class TemperatureDataset(Dataset):
    def __init__(self, data):
        self.X = torch.tensor(data.drop(columns=['Temperature', 'Season']).values, dtype=torch.float32)
        self.y = torch.tensor(data['Temperature'].values, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TemperatureDataset(train_data)
val_dataset = TemperatureDataset(val_data)

print(train_data.head(5))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

class TemperatureNN(nn.Module):
    def __init__(self):
        super(TemperatureNN, self).__init__()
        self.fc1 = nn.Linear(11, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

      Temperature  Humidity  Wind Speed  Precipitation (%)  Cloud Cover  \
3958         32.0        66         6.0               92.0            2   
2239         15.0        96        11.0               57.0            4   
3608         28.0        58         6.5               11.0            1   
6848         47.0        84        44.5               85.0            4   
6119         18.0        66        14.0               10.0            2   

      Atmospheric Pressure  UV Index  Season  Visibility (km)  Weather Type  \
3958              1.117800         2  Spring              1.5             3   
2239              1.299791         0  Autumn              4.0             3   
3608              1.525331        10  Summer              9.0             1   
6848              0.635268         2  Winter              2.0             3   
6119              0.844410         1  Autumn              8.0             2   

      Location_coastal  Location_inland  Location_mountain  
3958         

In [12]:
#season 제외한 모든 Columns 사용해서 기온 예측 살행

model = TemperatureNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)  # MSE 계산
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)  # 평균 계산

    # 검증 루프
    model.eval()
    val_loss = 0
    num_batches = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)  # MSE 계산
            val_loss += loss.item()  # 배치 손실 누적
    val_loss /= len(val_loader)  # 검증 손실 평균 계산

    # 출력
    if (epoch + 1) % 20 == 0:
      print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

print(f"Total Training Time: {time.time() - start_time:.2f} seconds")

Epoch 20/200, Train Loss: 121.5645, Validation Loss: 113.1695
Epoch 40/200, Train Loss: 111.9551, Validation Loss: 119.4001
Epoch 60/200, Train Loss: 103.9941, Validation Loss: 118.6580
Epoch 80/200, Train Loss: 95.5536, Validation Loss: 129.8151
Epoch 100/200, Train Loss: 88.5739, Validation Loss: 132.5675
Epoch 120/200, Train Loss: 82.9167, Validation Loss: 135.3095
Epoch 140/200, Train Loss: 76.8261, Validation Loss: 136.5946
Epoch 160/200, Train Loss: 73.9993, Validation Loss: 138.5300
Epoch 180/200, Train Loss: 70.5857, Validation Loss: 138.5064
Epoch 200/200, Train Loss: 68.4863, Validation Loss: 136.7414
Total Training Time: 315.54 seconds
